In [2]:
class Player:
    def __init__(self, health, gold, experience, weapons, armor, skills, elementalResistances):
        self.health = health + armor.health
        self.gold = gold
        self.weapons = weapons
        self.armor = armor
        self.experience = experience
        self.skillList = skills 
        self.criticalRate = 0.05 + weapons.criticalRate;
        self.attackDamage = 10 + weapons.attackDamage
        self.magicDamage = 10 + weapons.magicDamage
        self.physicalDefense = 10 + armor.physicalDefense
        self.magicDefense = 10 + armor.magicDefense
        self.luck = 0.05 + armor.luck
        self.elementalResistances = {'Fire':0+armor.elementalResistances['Fire'], 
                                     'Water':0+armor.elementalResistances['Water'], 
                                     'Electric':0+armor.elementalResistances['Electric'],
                                     'Ice':0+armor.elementalResistances['Ice'],
                                     'Dark':0+armor.elementalResistances['Dark'],
                                     'Light':0+armor.elementalResistances['Light'],
                                     'Physical':0+armor.elementalResistances['Physical']
                                    }
    
    def adjustHealth(self, health): #Health can be positive or negative 
        self.health += health
    
    def changeWeapon(self, weapon): 
        self.attackDamage = 10 + weapons.attackDamage + buffs.attackDamage - debuffs.attackDamage
    
    def adjustattackDamage(self, value):
        self.attackDamage += value

    def adjustmagicDamage(self, value):
        self.magicDamage += value
    
    def printElementalResistances(self):
        for y in self.elementalResistances[x]:
            print (y,':',self.elementalResistances[x][y])
        
        
        

In [3]:
class Weapon:
    def __init__(self, attackDamage, magicDamage, criticalRate, element):
        self.attackDamage = attackDamage 
        self.magicDamage = magicDamage 
        self.criticalRate = criticalRate
        self.element = element

class Armor:
    def __init__(self, health, physicalDefense, magicDefense, luck, elementalResistances):
        self.physicalDefense = physicalDefense
        self.health = health
        self.magicDefense = magicDefense 
        self.luck = luck 
        self.elementalResistances = elementalResistances


In [4]:
class Enemy:
    def __init__(self, health, name, goldReward, experienceReward, attackDamage, magicDamage, 
                 physicalDefense, magicDefense, elementalResistances):
        #elemental resistances could be a dictionary with elements and respective resistances 
        self.health = health
        self.name = name
        self.goldReward = goldReward
        self.experienceReward = experienceReward
        self.attackDamage = attackDamage
        self.magicDamage = magicDamage
        self.physicalDefense = physicalDefense
        self.magicDefense = magicDefense 
        self.elementalResistances = elementalResistances
    
    def adjustHealth(self, health):
        self.health += health



In [5]:
from random import randint
import random
from IPython.display import clear_output
class Skill:
    def __init__(self, name, baseDmg, variationDmg, energyCost, element, skillType):

        self.name = name
        self.baseDmg = baseDmg
        self.variationDmg = variationDmg 
        self.energyCost = energyCost
        self.element = element 
        self.skillType = skillType
    
    def executeSkill(self, target, user): #skill type refers to physical, magical, or neutral damage 
        rawDamage = randint(self.baseDmg-self.variationDmg, self.baseDmg+self.variationDmg) #establish lower and upper bounds of damage 
        if (self.skillType == 'Physical'):
            overallDamage = rawDamage + user.attackDamage - target.physicalDefense - target.elementalResistances[self.element]
            if (random.uniform(0, 1) <= user.criticalRate):
                overallDamage *= 1.5
                print("Critical Strike!")
            if (target.elementalResistances[self.element] > 5):
                print("Target is resistant to this element!")
            if (target.elementalResistances[self.element] < 0):
                print("Target is weak to this element!")
            if (overallDamage < 0):
                overallDamage = 0 #this is to make sure the damage does not go negative before the next line 
            target.adjustHealth(-1 * overallDamage)
            if (isinstance(user, Player)): #adjust print statement based on if the user was the player or the enemy
                print("You dealt " + str(overallDamage) + " damage to the enemy " + target.name + "!" )
            if (isinstance(user, Enemy)):
                print("The enemy dealt " + str(overallDamage) + " damage to you!" )                
            
        if (self.skillType == 'Magical'):
            overallDamage = rawDamage + user.magicDamage - target.magicDefense - target.elementalResistances[self.element]
            if (target.elementalResistances[self.element] > 5):
                print("Target is resistant to this element!")
            if (target.elementalResistances[self.element] < 0):
                print("Target is weak to this element!")
            if (overallDamage < 0):
                overallDamage = 0 #this is to make sure the damage does not go negative before the next line
            target.adjustHealth(-1 * overallDamage)
            if (isinstance(user, Player)): #adjust print statement based on if the user was the player or the enemy
                print("You dealt " + str(overallDamage) + " damage to the enemy " + target.name + "!" )
            if (isinstance(user, Enemy)):
                print("The enemy dealt " + str(overallDamage) + " damage to you!" )   
            ##put a function here that checks the health for game overs and/or enemy defeats 
            #or just check after we call this function 
        #clear_output()
        return overallDamage 
    
class BuffDebuffSkill:
    def __init__(self, stat, adjustment, skillType):
        self.stat = stat #tracks which stat in particular will be adjusted 
        self.skillType = skillType #tracks how much the stat gets adjusted 
        if (isinstance(self.skillType, 'Buff')):
            self.adjustment = adjustment 
        else:
            self.adjustment = adjustment * -1 
    
    def executeSkill(self, target, user):
        if (self.stat == "Attack Damage"):
            target.adjustattackDamage(self.adjustment)
        if (self.stat == "Magic Damage"):
            target.adjustmagicDamage(self.adjustment)        
        
            
        
        

In [58]:
baseDmg = 3
variationDmg = 1
randint(baseDmg-variationDmg, baseDmg+variationDmg)

3

In [70]:
elementResistance = {'Fire':3, 'Water': 2, 'Electric': 0, 'Ice': 5, 'Dark': 3, 'Light': -1, 'Physical': 0}
scythe = Weapon(3, 3, 0.01, 'Physical')
plating = Armor(15, 1, 1, 1, elementResistance)
mainCharacter = Player(100, 0, 0, scythe, plating, 0, elementResistance)
slimeEnemy = Enemy(50, 'Slime', 10, 10, 5, 5, 3, 3, elementResistance)
fireballSpell = Skill(7.5, 2.5, 8, 'Fire', 'Magical')
attack = Skill(3,1,0,'Physical', 'Physical')

In [72]:
attack.executeSkill(slimeEnemy, mainCharacter)

You dealt 12 damage to the enemy Slime!


12

In [ ]:
class Weapon:
    def __init__(self, attackDamage, magicDamage, criticalRate, element):
        

In [5]:
elementResistance = {'Fire':3, 'Water': 2} ##example for element resistance 

In [6]:
elementResistance['Fire']

3

In [13]:
print("hi")
print("ox")

hi
ox


In [8]:
import json
unlockables = {}
unlockables['skillUnlocks'] = [] ##so one entry in the unlockables dictionary is a list of skills that are unlocked 
unlockables['skillUnlocks'].append('Fireball')
with open('unlockables.txt', 'w') as outfile:
    json.dump(unlockables, outfile)